1. Load the html data

In [1]:
import glob

In [2]:
html_files = glob.glob(r'C:\Users\paras\Desktop\Project\*.html')

In [3]:
import codecs

In [4]:
s=" "
html_array =[]

for file in html_files:
    f = codecs.open(file, 'r')
    html_array.append(f.read())

In [5]:
html = s.join(html_array)

In [6]:
html.count("html")

3265

2.parse them using BeautifulSoup


In [7]:
from bs4 import BeautifulSoup

In [8]:
parsed_html = BeautifulSoup(html,"html.parser")


In [9]:
reviews = parsed_html.find_all('div',attrs = {'class':"text_here review-desc-more",'itemprop':"description"})

In [10]:
ratings = parsed_html.find_all('div',attrs = {'class':'rating-section review-user-score'} )

In [11]:
import re

In [12]:
def cleanhtml(raw_html):
    clean = re.compile('<.*?>')
    clean_text = re.sub(clean,'',raw_html)
    return clean_text
    

In [13]:
revws =[]
for i in range(0,len(reviews)):
    revws.append(cleanhtml(reviews[i].text))

In [14]:
def fun_rat(rating):
    clean = re.compile('<.*?>')
    clean_text = re.sub(clean,'',rating)
    clean_text = re.sub('\n','',clean_text)
    #clean_text = re.sub(' ','',clean_text)
    clean_text = re.sub(r'[^\.\w\S]','',clean_text)
    clean_text=clean_text[3:6]
    return clean_text
    
    
    
    

In [15]:
rat = []
for i in range(0,len(ratings)):
    rat.append(fun_rat(ratings[i].text))

In [16]:
rat[:10]

['5.0', '5.0', '5.0', '4.0', '4.0', '5.0', '1.0', '5.0', '5.0', '5.0']

3. Convert the data into dataframe

In [17]:
import pandas as pd

In [18]:
Reviews = pd.DataFrame({'Reviews':revws,'Ratings':rat})

In [19]:
Reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
Reviews    200 non-null object
Ratings    200 non-null object
dtypes: object(2)
memory usage: 3.2+ KB


In [20]:
Reviews.columns

Index(['Reviews', 'Ratings'], dtype='object')

In [21]:
Reviews['Reviews'][0]

'\n                                I have taken personal loan through Ujjivan Small Finance .The rate of interest and the processing fee was quite nominal.This loan was sanctioned within 3 to 4 days .The loan process was clearly shared by the agent .I am paying the premium on EMI basis and everything was good to me so far .'

In [22]:
# Remove word stems using a Porter stemmer
import nltk
ps = nltk.PorterStemmer()

    
Reviews['Reviews'] = Reviews['Reviews'] .apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))    
 

In [23]:
Reviews

,Reviews,Ratings
0,I have taken person loan through ujjivan small...,5.0
1,I have taken person loan directli through capi...,5.0
2,My person loan journey start with jana small f...,5.0
3,I have taken a person loan from icici bank and...,4.0
4,"took my person loan from bajaj finance, onc i ...",4.0
...,...,...
195,I have appli a person loan directli from shrir...,4.0
196,I have appli for hdfc bank person loan through...,4.5
197,"the rate of interest wa 18.99%, it wa quit nom...",5.0
198,I have avail person loan through hdfc bank .th...,5.0


In [24]:
Reviews = Reviews.replace({r'\s+$':'',r'^\s+': ''},regex=True).replace(r'\n',' ',regex=True)

In [25]:
Reviews['Reviews'][1]

'I have taken person loan directli through capit first sinc got the offer.th rate of interest and the process fee wa veri low and the agent gave a clear pictur about the loan process.i have paid emi for 10 month .thi loan wa taken for famili marriag purpose.'

In [26]:
Reviews.drop_duplicates(inplace=True)

In [27]:
Reviews.count()

Reviews    145
Ratings    145
dtype: int64

In [28]:
Reviews.head()

,Reviews,Ratings
0,I have taken person loan through ujjivan small...,5.0
1,I have taken person loan directli through capi...,5.0
2,My person loan journey start with jana small f...,5.0
3,I have taken a person loan from icici bank and...,4.0
4,"took my person loan from bajaj finance, onc i ...",4.0


In [29]:
# change words to lower case - Hello, HELLO, hello are all the same word

Reviews['Reviews'] = Reviews['Reviews'].str.lower()
print(Reviews['Reviews'])

0      i have taken person loan through ujjivan small...
1      i have taken person loan directli through capi...
2      my person loan journey start with jana small f...
3      i have taken a person loan from icici bank and...
4      took my person loan from bajaj finance, onc i ...
                             ...                        
195    i have appli a person loan directli from shrir...
196    i have appli for hdfc bank person loan through...
197    the rate of interest wa 18.99%, it wa quit nom...
198    i have avail person loan through hdfc bank .th...
199    i have appli for indusind bank person loan thr...
Name: Reviews, Length: 145, dtype: object


In [30]:
# Remove punctuation
Reviews['Reviews'] = Reviews['Reviews'].str.replace(r'[^\w\d\s]', ' ')

In [31]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))


Reviews['Reviews'] = Reviews['Reviews'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))



In [32]:
Reviews['Reviews']

0      taken person loan ujjivan small financ rate in...
1      taken person loan directli capit first sinc go...
2      person loan journey start jana small financ ba...
3      taken person loan icici bank loan amount wa rs...
4      took person loan bajaj finance onc finish cons...
                             ...                        
195    appli person loan directli shriram finance loa...
196    appli hdfc bank person loan onlin avail thi lo...
197    rate interest wa 18 99 wa quit nomin accord go...
198    avail person loan hdfc bank rate interest proc...
199    appli indusind bank person loan onlin avail th...
Name: Reviews, Length: 145, dtype: object

In [33]:
Reviews.dtypes

Reviews    object
Ratings    object
dtype: object

In [34]:
Reviews['Ratings'] = Reviews['Ratings'].astype(float)

In [35]:
Reviews.columns

Index(['Reviews', 'Ratings'], dtype='object')

Calculate Sentimate

In [36]:
def cal_senti(Reviews):
    if Reviews['Ratings'] < 4.0:
        return 1  #Negative
    else:
        return 0  #Positive

In [37]:
Reviews['Sentimate'] = Reviews.apply(cal_senti,axis=1)

In [38]:
Reviews.head()

,Reviews,Ratings,Sentimate
0,taken person loan ujjivan small financ rate in...,5.0,0
1,taken person loan directli capit first sinc go...,5.0,0
2,person loan journey start jana small financ ba...,5.0,0
3,taken person loan icici bank loan amount wa rs...,4.0,0
4,took person loan bajaj finance onc finish cons...,4.0,0


In [39]:
y,text  =Reviews.Sentimate, Reviews.Reviews

In [40]:
text[0]

'taken person loan ujjivan small financ rate interest process fee wa quit nominal thi loan wa sanction within 3 4 day loan process wa clearli share agent pay premium emi basi everyth wa good far'

Split the data into train and test at ratio of 70:30

In [41]:
text[0]

'taken person loan ujjivan small financ rate interest process fee wa quit nominal thi loan wa sanction within 3 4 day loan process wa clearli share agent pay premium emi basi everyth wa good far'

In [42]:
from nltk.tokenize import word_tokenize
import nltk

all_words = []

for message in text:
    word = word_tokenize(message)
    for w in word:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)        

In [43]:
type(all_words)

nltk.probability.FreqDist

In [44]:
all_words

FreqDist({'loan': 401, 'wa': 275, 'person': 149, 'interest': 143, 'rate': 137, 'thi': 132, 'process': 107, 'bank': 107, 'amount': 105, 'rs': 93, ...})

In [45]:
# print the total number of 15 most words

print("Number of most 15 common words :{}".format(all_words.most_common(15)))

print("length of total word :{}".format(len(all_words)))

Number of most 15 common words :[('loan', 401), ('wa', 275), ('person', 149), ('interest', 143), ('rate', 137), ('thi', 132), ('process', 107), ('bank', 107), ('amount', 105), ('rs', 93), ('taken', 84), ('pay', 68), ('veri', 67), ('appli', 66), ('emi', 65)]
length of total word :663


In [46]:
# use the 50 most common word

word_features = list(all_words.most_common(200))

In [47]:
type(word_features)

list

In [48]:
#word_features

In [49]:
def find_features(message):
    words = word_tokenize(message)
    features = {} 
    for i in word_features:
        features[i[0]] = (i[0] in words)
        
    return features    

In [50]:
#features

In [51]:
text[0]

'taken person loan ujjivan small financ rate interest process fee wa quit nominal thi loan wa sanction within 3 4 day loan process wa clearli share agent pay premium emi basi everyth wa good far'

In [52]:
#features = find_features(text[0])
#for key, value in features.items():
    #print(key)

In [53]:
#features = find_features(text[0])
#for key, value in features.items():
 #   if value == True:
  #      print(key)

    words = word_tokenize(text[5])
    features_2 = {} 
    #for i in word_features:
        #features_2[i[0]] = (i[0] in words)

In [54]:
import numpy as np

In [55]:
# Now lets do it for all the messages
text = list(zip(text, y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(text)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in text]

In [56]:
text[0]

('taken person loan edelweiss retail financ limit online due date wa 1st everi month thi chang 10th everi month helpful loan wa got sanction time satisfi servic',
 1)

In [57]:
# we can split the featuresets into training and testing datasets using sklearn
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [58]:
len(training)

108

In [59]:
len(testing)

37

In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [61]:
#define model to train
names = ['K Nearest neighbours','Decision Tree','Rendom Forest','Logistic Regression','SGD Classfier','Navie Bayes','SVM Linnear']

classfier = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(max_iter=100),
    SGDClassifier(),
    SVC(kernel='linear')
]

models = list(zip(names,classfier))

print(models)

[('K Nearest neighbours', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')), ('Decision Tree', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')), ('Rendom Forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_lea

In [62]:
# wrap models in NLTK
from nltk.classify.scikitlearn import SklearnClassifier

for name,model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model,testing)*100
    print('{}: Accuracy: {}'.format(name,accuracy))

K Nearest neighbours: Accuracy: 86.48648648648648
Decision Tree: Accuracy: 70.27027027027027
Rendom Forest: Accuracy: 83.78378378378379

C:\Users\paras\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\paras\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



Logistic Regression: Accuracy: 86.48648648648648
SGD Classfier: Accuracy: 83.78378378378379
Navie Bayes: Accuracy: 78.37837837837837


In [63]:
# Ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 78.37837837837837


In [64]:
# make class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [65]:
# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])

              precision    recall  f1-score   support

           0       0.88      0.94      0.91        31
           1       0.50      0.33      0.40         6

    accuracy                           0.84        37
   macro avg       0.69      0.63      0.65        37
weighted avg       0.82      0.84      0.82        37



predicted     
                  ham spam
actual ham         29    2
       spam         4    2

In [66]:
confusion_matrix(labels, prediction)

array([[29,  2],
       [ 4,  2]], dtype=int64)

In [67]:
from sklearn.metrics import confusion_matrix